In [ ]:
import os
import pandas as pd
import re
from rapidfuzz import process, fuzz

# Function to normalize text
def normalize_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        text = re.sub(r'\b(feat|ft|featuring|remix|version)\b', '', text)  # Remove problematic terms
    return text

# Load the CSV files
spotify_csv_path = r'D:\QuintoSemestre\ETL\Grammy_Analysis\data\spotify_updated.csv'
grammys_csv_path = r'D:\QuintoSemestre\ETL\Grammy_Analysis\data\grammys_updated_with_id.csv'  # Using the file with spotify_id

spotify_merge = pd.read_csv(spotify_csv_path)
grammy_merge = pd.read_csv(grammys_csv_path)

# Normalize the artist, song, and album names in both datasets
grammy_merge['artist_clean'] = grammy_merge['artist'].apply(normalize_text)
grammy_merge['nominee_clean'] = grammy_merge['nominee'].apply(normalize_text)
spotify_merge['artists_clean'] = spotify_merge['artists'].apply(normalize_text)
spotify_merge['track_name_clean'] = spotify_merge['track_name'].apply(normalize_text)
spotify_merge['album_name_clean'] = spotify_merge['album_name'].apply(normalize_text)

# Explode so that each artist in a list has its own row
spotify_merge = spotify_merge.explode('artists_clean')

# Direct merge based on 'spotify_id' and 'track_id'
merged_df = pd.merge(grammy_merge, spotify_merge, left_on='spotify_id', right_on='track_id', how='left')

# Find rows without a match in the merge based on spotify_id
no_match_df = merged_df[merged_df['track_id'].isnull()]

# If there are rows with no match, perform fuzzy matching
song_threshold = 70  # Threshold
artist_threshold = 70
album_threshold = 60

matches = []
for i, row in no_match_df.iterrows():
    song_match = process.extractOne(row['nominee_clean'], spotify_merge['track_name_clean'], scorer=fuzz.ratio)
    
    # Look for a match with track_name
    if song_match and song_match[1] >= song_threshold:
        matched_rows = spotify_merge[spotify_merge['track_name_clean'] == song_match[0]]
        artist_matches = []
        
        for _, matched_row in matched_rows.iterrows():
            artist_match = process.extractOne(row['artist_clean'], [matched_row['artists_clean']], scorer=fuzz.ratio)
            if artist_match and artist_match[1] >= artist_threshold:
                artist_matches.append(matched_row)  # Only add the full row
            
        if artist_matches:
            matches.extend(artist_matches)
        else:
            matches.append(row)
    else:
        # Look for a match with album_name if no match with track_name
        album_match = process.extractOne(row['nominee_clean'], spotify_merge['album_name_clean'], scorer=fuzz.ratio)
        if album_match and album_match[1] >= album_threshold:
            matched_rows = spotify_merge[spotify_merge['album_name_clean'] == album_match[0]]
            artist_matches = []
            
            for _, matched_row in matched_rows.iterrows():
                artist_match = process.extractOne(row['artist_clean'], [matched_row['artists_clean']], scorer=fuzz.ratio)
                if artist_match and artist_match[1] >= artist_threshold:
                    artist_matches.append(matched_row)  # Only add the full row
            
            if artist_matches:
                matches.extend(artist_matches)
            else:
                matches.append(row)
        else:
            matches.append(row)
    
    # Show progress every 100 rows
    if (i + 1) % 100 == 0:
        print(f"Processed {i + 1} rows out of {len(no_match_df)}")

# Create a DataFrame with the matches
fuzzy_matched_df = pd.DataFrame(matches)

# Concatenate the results of the direct merge and the fuzzy matches
final_merged_df = pd.concat([merged_df.reset_index(drop=True), fuzzy_matched_df.reset_index(drop=True)], ignore_index=True)

# Drop auxiliary columns except 'spotify_id'
final_merged_df = final_merged_df.drop(columns=['artist_clean', 'nominee_clean', 'artists_clean', 'track_name_clean', 'album_name_clean'], errors='ignore')

# Ensure that the 'spotify_id' column remains
if 'spotify_id' not in final_merged_df.columns:
    print("The 'spotify_id' column is not present in the final DataFrame.")
else:
    print("The 'spotify_id' column is present in the final DataFrame.")

# Remove duplicates
final_merged_df = final_merged_df.drop_duplicates(subset=['nominee', 'artist', 'category'])

# Check the number of rows after the merge
merged_count = final_merged_df.shape[0]
print(f"Number of rows after the merge: {merged_count}")

# Optional: Verify the first rows of the cleaned DataFrame
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
final_merged_df.head(20)

# Optional: Save the resulting DataFrame to a CSV file
output_path = r'D:\QuintoSemestre\ETL\Grammy_Analysis\data\final_merged_spotify_grammys.csv'
final_merged_df.to_csv(output_path, index=False)
print(f"Merge completed and file saved as '{output_path}'.")


In [1]:
import requests
import base64
import os

# Set your Spotify client credentials
client_id = os.getenv('SPOTIFY_CLIENT_ID')  # Add your credentials
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')  # Add your credentials

def get_spotify_access_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"

    # Encode credentials in base64
    auth_string = f"{client_id}:{client_secret}"
    b64_auth_string = base64.b64encode(auth_string.encode()).decode()

    headers = {
        "Authorization": f"Basic {b64_auth_string}"
    }
    data = {
        "grant_type": "client_credentials"
    }

    # Make the POST request to get the access token
    response = requests.post(url, headers=headers, data=data)

    if response.status_code == 200:
        token_info = response.json()
        access_token = token_info['access_token']
        print(f"Access Token: {access_token}")
        return access_token
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Get the Spotify access token
token = get_spotify_access_token(client_id, client_secret)


Access Token: BQAMEWA1LgPkH9YRxy7G6yQKv0FSCYhd9nrey-FioLBNu22gUiBG2yluz8ZC5TKLDelTkaZcahrY2wgGEnIyHibWBSR0u_rXe0lDYWUSS2KuHMXYiyg


In [1]:
import pandas as pd
import requests
import time

# Path to the CSV file
csv_path = r'D:\QuintoSemestre\ETL\Grammy_Analysis\data\final_merged_spotify_grammys.csv'

# Load the CSV file
final_merged_df = pd.read_csv(csv_path)

# Check if the required columns exist
required_columns = ['track_id', 'spotify_id']
for col in required_columns:
    if col not in final_merged_df.columns:
        raise ValueError(f"The column '{col}' is not found in the DataFrame.")

# Define a function to query the Spotify API (track information)
def get_spotify_info(spotify_id, access_token):
    url = f"https://api.spotify.com/v1/tracks/{spotify_id}"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error if the response is not 200
        return response.json()  # Return track data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching track information for {spotify_id}: {e}")
        return None  # Error handling

# Define a function to query the Spotify API (audio features)
def get_audio_features(spotify_id, access_token, retries=5, backoff_factor=2):
    url = f"https://api.spotify.com/v1/audio-features/{spotify_id}"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    for i in range(retries):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raise an error if the response is not 200
            return response.json()  # Return audio features
        except requests.exceptions.HTTPError as http_err:
            if response.status_code == 429:  # If 'Too Many Requests' status is received
                retry_after = int(response.headers.get("Retry-After", backoff_factor))
                wait_time = retry_after * (backoff_factor ** i)
                print(f"Too many requests. Waiting {wait_time} seconds before retrying...")
                time.sleep(wait_time)
            else:
                print(f"Error fetching audio features for {spotify_id}: {http_err}")
                return None  # Error handling
        except requests.exceptions.RequestException as e:
            print(f"Error fetching audio features for {spotify_id}: {e}")
            return None
    return None  # If after multiple attempts, the response can't be obtained

# Access token (you need to update this with your valid token)
ACCESS_TOKEN = 'BQAMEWA1LgPkH9YRxy7G6yQKv0FSCYhd9nrey-FioLBNu22gUiBG2yluz8ZC5TKLDelTkaZcahrY2wgGEnIyHibWBSR0u_rXe0lDYWUSS2KuHMXYiyg'

# Limit API requests (Spotify recommends a max of 50-100 per second, depending on the API)
REQUEST_DELAY = 0.5  # Half-second delay between each request

# Iterate over the DataFrame to fill NaNs
for index, row in final_merged_df[final_merged_df['track_id'].isnull()].iterrows():
    spotify_id = row['spotify_id']  # Ensure this column exists
    
    if pd.notna(spotify_id):  # Check if spotify_id is not NaN
        try:
            # Fetch track information from Spotify
            track_info = get_spotify_info(spotify_id, ACCESS_TOKEN)
            
            if track_info:
                # Fill in relevant columns
                final_merged_df.loc[index, 'track_id'] = track_info.get('id')
                final_merged_df.loc[index, 'artists'] = ', '.join([artist['name'] for artist in track_info.get('artists', [])])
                final_merged_df.loc[index, 'album_name'] = track_info.get('album', {}).get('name')
                final_merged_df.loc[index, 'track_name'] = track_info.get('name')
                final_merged_df.loc[index, 'popularity'] = track_info.get('popularity')
                final_merged_df.loc[index, 'duration_ms'] = track_info.get('duration_ms')
            
                # Fetch audio features
                audio_features = get_audio_features(spotify_id, ACCESS_TOKEN)
                
                if audio_features:
                    final_merged_df.loc[index, 'danceability'] = audio_features.get('danceability')
                    final_merged_df.loc[index, 'energy'] = audio_features.get('energy')
                    final_merged_df.loc[index, 'tempo'] = audio_features.get('tempo')
                    final_merged_df.loc[index, 'track_genre'] = track_info.get('genre', None)  # If genre is available
                
            # Show progress every 100 rows
            if index % 100 == 0:
                print(f"Information filled in row {index}")
        
        except Exception as e:
            print(f"Error processing row {index}: {e}")

        # Add a delay between each request to avoid overwhelming the API
        time.sleep(REQUEST_DELAY)

# Check how many rows still have NaN after processing
remaining_nan_counts = final_merged_df.isnull().sum()
print("NaN count after processing:")
print(remaining_nan_counts)

# Optional: Verify the first rows of the cleaned DataFrame
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(final_merged_df.head(20))  # Ensure to print the DataFrame

# Optionally save the updated result to a new CSV file
# output_path = r'C:\Users\USUARIO\Documents\Workshop2\notebooks\Merged_Updated.csv'
# final_merged_df.to_csv(output_path, index=False)
# print(f"Updated file saved to: {output_path}")


Información rellenada en fila 100
Información rellenada en fila 200
Información rellenada en fila 300
Error al obtener información del track 1ZwdS5xdxEREPySFridCfh: 404 Client Error: Not Found for url: https://api.spotify.com/v1/tracks/1ZwdS5xdxEREPySFridCfh
Información rellenada en fila 400
Información rellenada en fila 500
Información rellenada en fila 600
Información rellenada en fila 700
Información rellenada en fila 800
Información rellenada en fila 900
Información rellenada en fila 1000
Información rellenada en fila 1100
Información rellenada en fila 1200
Información rellenada en fila 1300
Información rellenada en fila 1400
Información rellenada en fila 1500
Información rellenada en fila 1600
Información rellenada en fila 1700
Información rellenada en fila 1800
Información rellenada en fila 1900
Demasiadas solicitudes. Esperando 2 segundos antes de reintentar...
Demasiadas solicitudes. Esperando 4 segundos antes de reintentar...
Demasiadas solicitudes. Esperando 8 segundos antes 

In [4]:
import pandas as pd


# Opcional: Verificar las primeras filas del DataFrame limpio
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
final_merged_df.head(5)

,id_x,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner,spotify_id,id_y,track_id,artists,album_name,track_name,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,duration_sec,id,duration_ms
0,1.0,2019.0,62nd Annual GRAMMY Awards (2019),2020-05-19 05:10:28,2020-05-19 05:10:28,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True,2Fxmhks0bxGSBdJ92vM42m,30867.0,2Fxmhks0bxGSBdJ92vM42m,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",bad guy,84.0,False,0.701,0.425,7.0,-10.965,1.0,0.375,0.328,0.13,0.1000,0.562,135.128,4.0,electro,194.087,NaN,NaN
1,2.0,2019.0,62nd Annual GRAMMY Awards (2019),2020-05-19 05:10:28,2020-05-19 05:10:28,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True,0RstfX9nRY1Lfuy1808MoT,NaN,0RstfX9nRY1Lfuy1808MoT,Bon Iver,"i,i","Hey, Ma",58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216706.0
2,3.0,2019.0,62nd Annual GRAMMY Awards (2019),2020-05-19 05:10:28,2020-05-19 05:10:28,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True,6ocbgoVGwYJhOv1GgI9NsF,19913.0,6ocbgoVGwYJhOv1GgI9NsF,Ariana Grande,"thank u, next",7 rings,84.0,True,0.778,0.317,1.0,-10.732,0.0,0.334,0.592,0.00,0.0881,0.327,140.048,4.0,dance,178.626,NaN,NaN
3,4.0,2019.0,62nd Annual GRAMMY Awards (2019),2020-05-19 05:10:28,2020-05-19 05:10:28,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True,4IablJ6SqVNGY4vrseyKxu,NaN,4IablJ6SqVNGY4vrseyKxu,H.E.R.,Hard Place,Hard Place,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271733.0
4,5.0,2019.0,62nd Annual GRAMMY Awards (2019),2020-05-19 05:10:28,2020-05-19 05:10:28,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True,6g6A7qNhTfUgOSH7ROOxTD,NaN,6g6A7qNhTfUgOSH7ROOxTD,"Jason Derulo, 2 Chainz",Talk Dirty,Talk Dirty (feat. 2 Chainz),67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,177685.0


In [16]:
import pandas as pd

csv_path = r'D:\QuintoSemestre\ETL\Grammy_Analysis\data\Grammy_And_Spotify_Merged.csv'
final_merged_df = pd.read_csv(csv_path)

# Excluir las columnas 'Unnamed: 0' y 'track_genre' al verificar NaN
columns_to_exclude = ['Unnamed: 0', 'track_genre','danceability','energy','tempo']
filtered_df = final_merged_df.drop(columns=columns_to_exclude, errors='ignore')

#filtered_df['id'] = filtered_df['id'].astype(int)
#filtered_df['year'] = filtered_df['id'].astype(int)


# Ver el número total de filas y columnas en el DataFrame (sin excluir columnas)
total_filas, total_columnas = final_merged_df.shape
print(f"Total de filas: {total_filas}")
print(f"Total de columnas: {total_columnas}")

# Ver cuántas filas tienen NaN en alguna columna (excluyendo las columnas especificadas)
filas_con_nan = filtered_df.isna().any(axis=1).sum()
print(f"Filas con valores NaN (excluyendo 'Unnamed: 0' ,'track_genre' ,'danceability','energy'y'tempo'): {filas_con_nan}")

# Ver cuántas columnas tienen al menos un NaN (excluyendo las columnas especificadas)
columnas_con_nan = filtered_df.isna().any(axis=0).sum()
print(f"Columnas con valores NaN (excluyendo 'Unnamed: 0' y 'track_genre','danceability','energy'y'tempo'): {columnas_con_nan}")


Total de filas: 4758
Total de columnas: 15
Filas con valores NaN (excluyendo 'Unnamed: 0' ,'track_genre' ,'danceability','energy'y'tempo'): 3264
Columnas con valores NaN (excluyendo 'Unnamed: 0' y 'track_genre','danceability','energy'y'tempo'): 15


In [9]:
# Mostrar las primeras 10 filas que tienen NaN
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
filtered_df.head(10)

# Optional: Save the resulting DataFrame to a CSV file
output_path = r'D:\QuintoSemestre\ETL\Grammy_Analysis\data\Grammy_And_Spotify_Merged.csv'
filtered_df.to_csv(output_path, index=False)
print(f"Merge cleaned completed and file saved as '{output_path}'.")


Merge cleaned completed and file saved as 'D:\QuintoSemestre\ETL\Grammy_Analysis\data\Grammy_And_Spotify_Merged.csv'.
